 The returns for the European, Australian and Japanese assets must be adjusted for the currency returns in EUR, AUD, JPY.

In [1]:
# European assets
# 1.  MSE
# 2. IEAC





The portfolio is equally weighted: $\omega_i = \frac{1}{n}$, with the adjusted portfolio returns we obtain the portfolio return by a simple inner product.